In [1]:
!pip install -q bitsandbytes datasets accelerate loralib
!pip install -q git+https://github.com/huggingface/transformers.git@main git+https://github.com/huggingface/peft.git

  error: subprocess-exited-with-error
  
  git clone --filter=blob:none --quiet https://github.com/huggingface/transformers.git 'C:\Users\sayed\AppData\Local\Temp\pip-req-build-9dxgkhx3' did not run successfully.
  exit code: 128
  
  [10 lines of output]
  error: RPC failed; curl 92 HTTP/2 stream 3 was not closed cleanly: CANCEL (err 8)
  error: 5463 bytes of body are still expected
  fetch-pack: unexpected disconnect while reading sideband packet
  fatal: early EOF
  fatal: fetch-pack: invalid index-pack output
  fatal: could not fetch 484d62b46a87f43e3df240823bd9227d8ebd11bc from promisor remote
  You can inspect what was checked out with 'git status'
  and retry with 'git restore --source=HEAD :/'
  
  [end of output]
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
error: subprocess-exited-with-error

git clone --filter=blob:none --quiet https://github.com/huggingface/transformers.git 'C:\Users\sayed\AppData\Local\Temp\pip-req-build-9dxgkhx

In [2]:
import pandas as pd
import datasets
from datasets import Dataset, DatasetDict

import torch
from peft import PeftModel, PeftConfig
from transformers import AutoModelForCausalLM, AutoTokenizer

In [3]:
from huggingface_hub import notebook_login
notebook_login()

In [4]:
peft_model_id = "Ebad/bloom-7b1-lora-tagger-for-tweet-generation"
config = PeftConfig.from_pretrained(peft_model_id)
model = AutoModelForCausalLM.from_pretrained(config.base_model_name_or_path, return_dict=True, load_in_8bit=True, device_map='auto')
tokenizer = AutoTokenizer.from_pretrained(config.base_model_name_or_path)

# Load the Lora model
model = PeftModel.from_pretrained(model, peft_model_id)

adapter_config.json:   0%|          | 0.00/679 [00:00<?, ?B/s]

C:\Users\sayed\AppData\Local\Programs\Python\Python310\lib\site-packages\huggingface_hub\file_download.py:147: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\sayed\.cache\huggingface\hub\models--Ebad--bloom-7b1-lora-tagger-for-tweet-generation. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
The `load_in_4bit` and `load_in_8bit` arguments are deprecated and

adapter_model.safetensors:   0%|          | 0.00/8.73M [00:00<?, ?B/s]

In [5]:
df = pd.read_csv('test_df.csv', engine='python')

In [6]:
tds = Dataset.from_pandas(df)
data = DatasetDict()
data['test'] = tds

In [7]:
data['test'][0]['prediction']

'cbc with username CBCToronto posted a tweet at 2018-03-24 14:38:24 with an image that is arafed protestors hold signs in a city street with number of likes 97 that reads ->: Hundreds gather in Toronto in support of U.S. rally for gun control <hyperlink> <hyperlink>'

In [8]:
output = []

In [11]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cuda'

In [12]:
for i in range(0,500): #predictions
  txt = data['test'][i]['prediction']
  batch = tokenizer(txt , return_tensors='pt').to(device)
  with torch.amp.autocast('cuda'):
    output_tokens = model.generate(**batch, max_new_tokens=50)
    output.append(tokenizer.decode(output_tokens[0], skip_special_tokens=True))

In [13]:
output[0]

'cbc with username CBCToronto posted a tweet at 2018-03-24 14:38:24 with an image that is arafed protestors hold signs in a city street with number of likes 97 that reads ->: Hundreds gather in Toronto in support of U.S. rally for gun control <hyperlink> <hyperlink> <hyperlink> <hyperlink> <hyperlink> <hyperlink> <hyperlink> <hyperlink> <hyperlink> <hyperlink> <hyperlink> <hyperlink> <hyperlink> <hyperlink> <hyper'

In [14]:
dicti = {'content':output}

In [15]:
dff = pd.DataFrame(dicti)

In [16]:
for i in range(len(dff)):

  dff.at[i,'content'] = dff.at[i,'content'].split("->: ")[1]
  print(i)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [23]:
dff.to_csv('output_Qwen.csv')